In [1]:
from binance import Client
from csv import writer

import datetime
import pandas as pd
import numpy as np
import time
import ta
import math
import talib


import binance_credentials as bc


np.seterr(divide='ignore', invalid='ignore')
pd.set_option('display.max_rows', None)

In [2]:
client = Client(bc.key,bc.secret)

RSI_WINDOW = 8
DMI_WINDOW = 8
OVERSOLD_THRESHOLD = 20
OVERSOLD_THRESHOLD_2 = 30
OVERBOUGHT_THRESHOLD = 90
EMA_F_WINDOW = 5
EMA_S_WINDOW = 10
MA_TREND_WINDOW = 50


In [44]:
df = pd.DataFrame(columns=['symbol', 'previousChange'])
info = pd.DataFrame(client.get_ticker())
df = info[['symbol','priceChangePercent']]
df = df[df.symbol.str.contains('USDT')]


for i in range(0,3):
    data = pd.DataFrame(client.get_ticker())
    change = data[['priceChangePercent']]
    df[f"priceChangePercent_'{i}'"] = change
    time.sleep(45)

df.tail(20)

,symbol,priceChangePercent,priceChangePercent_'0',priceChangePercent_'1',priceChangePercent_'2'
1858,JOEUSDT,12.477,12.568,12.557,12.727
1877,ACHUSDT,-1.072,-1.072,-1.234,-0.769
1880,IMXUSDT,-3.109,-3.034,-2.794,-3.026
1883,GLMRUSDT,-3.841,-3.841,-3.678,-3.504
1899,LOKAUSDT,-15.981,-15.951,-15.616,-16.317
1906,SCRTUSDT,-0.568,-0.568,-0.421,-0.202
1909,API3USDT,-4.349,-4.349,-4.072,-4.087
1910,BTTCUSDT,4.245,4.739,4.717,4.245
1915,ACAUSDT,9.375,9.375,9.810,9.810
1918,ANCUSDT,-8.618,-8.618,-8.615,-8.508


In [3]:
def create_top():
    df = pd.DataFrame(client.get_ticker())
    df = df[['openTime','symbol','priceChange','priceChangePercent','lastPrice','prevClosePrice']]
    df[['priceChange','priceChangePercent','lastPrice','prevClosePrice']] = df[['priceChange','priceChangePercent','lastPrice','prevClosePrice']].astype(float)   
    df = df.sort_values(['priceChangePercent'], ascending=False)
    df = df[df.symbol.str.contains('USDT')]
    df = df[(df.lastPrice > 0.01) & (df.lastPrice < 15)]
    df = df[~(df.symbol.str.contains('UP|DOWN|BULL|BEAR|TUSD|GBP|BUSD|EUR|PAXG|USDC|AUD'))]
    df = df.head(5)
    df = df.reset_index(inplace=False) 
    return list(df.symbol.values[:])

In [4]:
create_top()

['ZILUSDT', 'LOKAUSDT', 'HIGHUSDT', 'VOXELUSDT', 'DARUSDT']

In [ ]:
def create_symbol_csv():
    info = client.get_exchange_info()
    symbols = [x['symbol'] for x in info['symbols']]
    exclude = ['UP','DOWN','BULL','BEAR','TUSD','GBP','BUSD','EUR','PAXG','USDC','AUD']
    non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
    relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]
    lower = [i.lower() for i in relevant]
    df = pd.DataFrame(lower, columns=['name'])
    df.to_csv('usdt_symbols.csv', index=False)
    # with open('usdt_symbols.csv','w') as file:
    #     w_obj = writer(file)
    #     w_obj.writerow(lower)
    #     file.close()
    # return lower
    return df

In [ ]:
create_symbol_csv()

In [ ]:
def write_csv(order, type):
    if type == 'B':
        order_list = ['BUY',order['symbol'],round(float(order['executedQty']),3),datetime.datetime.fromtimestamp(order['time']/1000).strftime('%Y-%m-%d %H:%M:%S'),order['price'],'DMI_RSI_MOMENTUM']
    elif type == 'S':
        order_list = ['SELL',order['symbol'],round(float(order['executedQty']),3),datetime.datetime.fromtimestamp(order['transactTime']/1000).strftime('%Y-%m-%d %H:%M:%S'),order['fills'][0]['price']]
    
    with open('trades.csv', 'a', newline='\n') as file:
        w_object = writer(file)
        w_object.writerow(order_list)
        file.close()

In [ ]:
def get_assets():
    print(f"Available funds:{round(float(client.get_account()['balances'][11]['free']),3)} USDT")
    return round(float(client.get_account()['balances'][11]['free'])/2,0)

In [ ]:
client.get_asset_balance(asset='BSW')

In [ ]:
def get_asset_open(asset):
    asset = asset[:-4]
    x = client.get_account()['balances']

    for i in x:
        if i['asset'] == asset:
            return float(i['free'])

In [ ]:
def apply_ta(data):
    rsi_indicator = ta.momentum.RSIIndicator(data.close, window=RSI_WINDOW)
    emafast_indicator = ta.trend.EMAIndicator(data.close, window=EMA_F_WINDOW)
    emaslow_indicator = ta.trend.EMAIndicator(data.close, window=EMA_S_WINDOW)
    smatrend_indicator = ta.trend.SMAIndicator(data.close, window=MA_TREND_WINDOW)
    macd_indicator = ta.trend.MACD(data.close, window_slow=6,window_fast=18,window_sign=12)
    dmi_indicator = ta.trend.ADXIndicator(data.high, data.low, data.close, window=DMI_WINDOW)

    data['ADX'] = dmi_indicator.adx()
    data['DM+'] = dmi_indicator.adx_pos()
    data['DM-'] = dmi_indicator.adx_neg()
    # data['EMA_SLOW'] = emaslow_indicator.ema_indicator()
    # data['EMA_FAST'] = emafast_indicator.ema_indicator()
    data['SMA_TREND'] = smatrend_indicator.sma_indicator()
   
    # data['MACD'] =macd_indicator.macd()
    # data['MACD_SIG'] = macd_indicator.macd_signal()
    # data['MACD_HST'] = macd_indicator.macd_diff()
    data['MOMENTUM'] = (data.close.pct_change()+1).cumprod()
    data['RSI'] = rsi_indicator.rsi()   
    
    return data

In [ ]:
def apply_ta(data):
    open = data.open
    high = data.high
    low = data.low
    close = data.close
    data['DCC'] = talib.CDLDARKCLOUDCOVER(open,high,low,close, penetration=0)
    data['HAMMER'] = talib.CDLHAMMER(open,high,low,close)
    data['INVHAMMER'] = talib.CDLINVERTEDHAMMER(open,high,low,close)
    data['ENGULF'] = talib.CDLENGULFING(open,high,low,close)
    data['PIERCING'] = talib.CDLPIERCING(open,high,low,close)
    data['THRUSTING'] = talib.CDLTHRUSTING(open,high,low,close)
    data['MORNING'] = talib.CDLMORNINGSTAR(open,high,low,close)
    data['MORNINGDOJI'] = talib.CDLMORNINGDOJISTAR(open,high,low,close)


    return data

In [ ]:
def sig_trigger(df):
    
    # df['ema_cross'] = np.where((df['EMA_FAST'].shift(1) < df['EMA_SLOW'].shift(1)) & (df['EMA_FAST'] > df['EMA_SLOW']),1,0)    
    # df['rsi_entry'] = np.where((df['RSI'].shift(2) <= OVERSOLD_THRESHOLD) & (df['RSI'].shift(1) <= OVERSOLD_THRESHOLD_2), 1, 0)
    # df['rsi_exit'] = np.where(df['RSI'] >= OVERBOUGHT_THRESHOLD, 1, 0)
    # df['signal_flip'] = np.where((df['MACD'].shift(1) > df['MACD_SIG'].shift(1)) & (df['MACD'] < df['MACD_SIG']),1,0)
    # df['dmi_cross'] = np.where((df['DM+'].shift(1) < df['DM-'].shift(1)) & (df['DM+'] > df['DM-']),1,0)
    # df['dmi_rsi'] = np.where((df['DM+'].shift(1) < df['DM-'].shift(1))  & (df['DM+'] > df['DM-']) & (df['RSI'] > 50),1,0 )
    # df['dmi_rsi_macd'] = np.where((df['DM+'].shift(2) < df['DM-'].shift(2))  & (df['DM+'].shift(1) > df['DM-'].shift(1)) & (df['RSI'].shift(1) > 50) & (df['signal_flip']),1,0 )
    df['uptrend'] = np.where((df['SMA_TREND'] < df.close), 1,0)
    df['adx_rsi_entry'] = np.where((df.ADX > 35) & (df.RSI < 50) & (df['DM+'] < df['DM-']),1,0)
    df['adx_rsi_exit'] = np.where((df.ADX < 35) & (df.RSI > 50) & (df['DM+'] > df['DM-']),1,0)
    # df['trigger'] = np.where((df['MOMENTUM'] >= 1), 1,0)
    # df['trigger'] = np.where((df['MACD_HST'].shift(1) > 0) & (df['MACD_HST'].shift(2) <= 0),1,0)
    # df['signal_flip'] = np.where((df['MACD'].shift(1) > df['MACD_SIG'].shift(1)) & (df['MACD'] < df['MACD_SIG']),1,0)

    return df
    

In [ ]:
def get_minutedata(symbol, interval, lookback, type=None):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, f'{lookback} hours ago UTC'))
    df = df.iloc[:,:6]
    df.columns = ['Time', 'open', 'high', 'low', 'close', 'volume']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    if (type == 'B'):
        df = apply_ta(df)
        # df = sig_trigger(df)
        # RSI buy and sell signals
        # conditions = [(df.adx_rsi_entry == 1), (df.adx_rsi_exit == 1)]
        # values = ['BUY','SELL']
        # df['trade'] = np.select(conditions, values)
    
    return df

In [ ]:
get_minutedata('ASRUSDT', '1m', '5', 'B')

In [ ]:
def strategy(asset, buy_amt, lb, period, SL=0.98, Target=1.05, open_position=False, pending_order=True):
    try:
        df = get_minutedata(asset, lb, period, type="B")
    except:
        print('Something went wrong! Hold on for 1 minute!')
        time.sleep(61)
        df = get_minutedata(asset, lb, period, type="B")
    
    if (df.trade[-1] == 'BUY'):
        print(asset)
        order = client.order_limit_buy(symbol=asset, quantity=round(buy_amt/df.close.iloc[-1]),price=df.open.iloc[-2])
        # order = client.create_order(symbol=asset, quantity=round(buy_amt/df.close.iloc[-1]),side='BUY',type='MARKET')

        while pending_order:
            open_order = client.get_order(symbol=asset, orderId=order['orderId'])
            if (open_order['status'] == 'FILLED'):
                open_position = True
                pending_order = False
                print(f"Order {open_order['orderId']} is filled!")
            elif (open_order['status'] == 'CANCELED'):
                pending_order = False
                break
            else:
                pending_order = True
                print(f"Order {open_order['orderId']} isn't filled!")

        buyprice = float(open_order['price'])
        print(f"{round(float(open_order['executedQty']),3)} {open_order['symbol']} bought for {buyprice} ")
        write_csv(open_order, type='B')
        if ((buyprice * SL) > df.low[-2]):
            stop_loss = df.low[-2]
        else:
            stop_loss = df.close[-1] * SL
        inc_sl = 1.01
        sell_qty = open_order['executedQty'] * 0.999
        while open_position:
            try:
                df = get_minutedata(asset, lb, '4')
            except:
                print('Something went wrong! Hold on for 1 minute!')
                time.sleep(61)
                df = get_minutedata(asset, lb, '4')
            
            if((buyprice*inc_sl) < df.low[-1]):
                stop_loss = buyprice * inc_sl
                inc_sl = inc_sl + 0.05

            print(f"Current close is: {str(df.close[-1])}")
            print(f"Aimed target is: {str(round(buyprice * Target,4))}")
            print(f"Aimed SL is: {str(round(stop_loss,4))}")
            time.sleep(30)
            if (df.close[-1] <= stop_loss) or (df.close[-1] > round(buyprice * Target,4)) or (df.trade[-1] == 'SELL'):
                try:
                    order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity=sell_qty)
                except:
                    order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity=math.floor(get_asset_open(asset)))
                sellprice = order['fills'][0]['price']
                print(f"{round(float(order['executedQty']),3)} {order['symbol']} sold for {sellprice} ")
                write_csv(order, type='S')
                break



            

In [ ]:
def running():
    coins = create_top()
    lookback = '1m'
    period = "3"
    
    assets = get_assets()
    count = 0
    
    while True:
        if count == 25:
            coins = create_top()
            print(coins)
            count = 0    
        for coin in coins:
            strategy(coin, assets, lookback, period)
        count += 1

        


In [ ]:
running()

In [ ]:
from binance.helpers import round_step_size

amount = 1111
tick_size = 0.00001
rounded_amount = round_step_size(amount, tick_size)